# 정선아(17011709), 문성용(17011741), 김소영(17011742)

# 1. PIP

### 1.1 기존 유사도 측정방법의 문제점

a. 데이터 sparsity에 따라 매우 제한된 수의 공동 평가. <br>
b. 공통 레이팅 항목 수가 1개일 경우, COR을 계산할 수 없고 COS은 1개의 상관없는 개인 레이팅 차이를 발생시킨다. <br>
c. 주어진 레이팅이 모두 <1, 1, 1>, <3, 3, 3>, <4, 4, 4>와 같은 flat한 상태라면 두 유저 사이의 COS은 측정될 수 없다. <br>
d. 두개의 벡터가 <2, 2>, <3, 3>과 같이 같은 선상에 존재하면 COS은 두개의 의미없는 차이를 발생시킨다. <br>
e. COR, COS는 종종 오류를 발생시킨다. 매우 먼 관계에 있는 유저들의 유사도가 가깝게 측정 될 수 있다. <br><br>
이와 같은 문제들은 레이팅의 수가 굉장히 제한적이거나 데이터셋이 sparse한 경우 더 심각한 문제를 불러 일으킬 수 있다. 왜냐하면 유사도 측정법 계산보다 우연에 의해 발생할 가능성이 더 크기 때문이다. 


### 1.2 새로운 유사도 측정법: PIP (Proximity-Impact-Popularity) 

a. 이 조치는 콜드 스타트 권장 조건에 더 효과적이기 위해 전통적인 유사성 또는 거리 측정만 채택하지 말고 데이터의 도메인 고유 의미를 활용한다. <br>
b. 좀 더 실용화하기 위해서는 방대한 재구축이나 추가 데이터 수집이 필요 없이 시스템의 유사성 측정법만 교체해 기존 CF 시스템에 쉽게 
   플러그인을 할 수 있도록 해야 한다. <br>
c. 이 조치는 새로운 사용자 콜드 스타트 조건에서 더 나은 결과를 보여줄 뿐만 아니라 콜드 스타트 이외의 다른 인기 있는 조치와 비교 가능한 결과를 보인다.

### 1.3 PIP 수식

PIP 측정법은 Proximity, Impact, Popularity의 3가지 요소로 이루어져 있다. PIP 측정법은 다음의 수식을 통해 u_i, u_j의 유사도를 측정한다. 

In [1]:
from IPython.display import Image

![title](pip수식.JPG)
a. r_ik, r_jk는 아이템 k에 대한 user i, user j의 rating이다. <br>
b. C_ij는 user i와 user j가 rating한 아이템의 집합니다. <br>
c. PIP(r_ik r_jk)는 r_ik, r_jk에 대한 PIP score이다. <br>
d. PIP(r1, r2) = Proximity(r1, r2) x Impact(r1, r2) x Popularity(r1, r2)

### 1.4 각 요소별 의미

a. Proximity <br>
![title](proximity.JPG)
(a1, a2), (b1, b2)모두 2라는 같은 거리값을 가지고 있다. 하지만 첫번째 쌍은 a1은 긍정적인 rating인 반면, a2는 부정적인 rating값이라서 이에 패널티가 부과되어야 한다.
반면 b1은 긍정적인 rating이고, b2는 중립이기때문에 패널티를 부과하지 않는다.

b. Impact
![image.png](impact.JPG)
(a1, a2), (b1, b2)모두 0의 거리값을 가진다. 하지만 첫번째 쌍은 더 강한 선호도 값에서 동의를 보여주므로 더 큰 영향을 미치는 것으로 여겨진다. 

c. Popularity
![image.png](popularity.JPG)
(a1, a2), (b1, b2) 모두 0의 거리값을 가지므로 같은 Impact요소를 가진다. 하지만 Mu를 모두 측정된 아이템의 평균이라고 할때, 둘 사이의 유사도는 Mu1의 평균을 같는 쌍보다 더 큰 값을 같는다. 

### 1.5 Agreement 함수 구현

Agreement 함수는 boolean function으로 여기서 r_max와 r_min은 레이팅의 최대 가능값, 최소 가능값을 의미한다. 

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing
from collections import OrderedDict 

In [3]:
def Agreement(r1, r2):
    if (r1 > r_med and r2 < r_med) or (r1 < r_med and r2 > r_med):
        return False
    
    return True

### 1.6 Proximity 함수 구현

Agreement(r1, r2)값의 True, False 두가지에 나눠 distance를 구한다. 

In [4]:
def Proximity(r1, r2):
    if Agreement(r1, r2):
        D = abs(r1 - r2)
    else:
        D = abs(r1 - r2) * 2
        
    return ((2 * (r_max - r_min) + 1) - D)**2

### 1.7 Impact 함수 구현

Agreement값에 따라 두가지 리턴 값을 갖는다. 

In [5]:
def Impact(r1, r2):
    if Agreement(r1, r2):
        return (abs(r1 - r_med) + 1) * (abs(r2 - r_med) + 1)
    else:
        return 1 / ((abs(r1 - r_med) + 1) * (abs(r2 - r_med) + 1))

### 1.8 Popularity 함수 구현

Mu_k는 아이템 k에 대한 모든 유저의 평균

In [6]:
def Popularity(r1, r2, k): # k는 아이템
    rating = np.array(MovieLens_pivot[k].values)
    #mu_k = np.nanmean(np.where(rating != 0, rating, np.nan))
    mu_k = rating.mean()
    if (r1 > mu_k and r2 > mu_k) or (r1 < mu_k and r2 < mu_k):
        return 1 + (((r1 + r2) / 2) - mu_k)**2
    else:
        return 1

### 1.9 PIP 구현

user i, user j가 레이팅한 모든 아이템에 대하여 PIP를 측정하고 SUM한다.

In [7]:
# 변수 초기화
r_max = 5
r_min = 1
r_med = (r_max + r_min) / 2

In [8]:
# PIP 함수 
def PIP(user1, user2):
    shared_item = []
    r1_item = np.array(MovieLens_pivot.loc[user1].values > 0, dtype = np.int)
    r2_item = np.array(MovieLens_pivot.loc[user2].values > 0, dtype = np.int)
    union = r1_item + r2_item
    for i in range(len(union)):
        if union[i] == 2:
            shared_item.append(MovieLens_pivot.columns[i])
    if len(shared_item) == 0:
        return 0
    
    sim_pip = 0
    for item in shared_item:
        r1 = MovieLens_pivot.loc[user1][item]
        r2 = MovieLens_pivot.loc[user2][item]
        sim_pip += Proximity(r1, r2) * Impact(r1, r2) * Popularity(r1, r2, item)
        
    return sim_pip

### 1.10 MovieLense Data를 이용하여 PIP Similarity 측정

In [9]:
import warnings
warnings.filterwarnings('ignore')

from UserSimilarity import MovieLens_pivot
# rating이 많은 사용자 1,000명, rating이 많은 영화 1,000개

In [19]:
# 시간이 너무 오래 걸리므로 축소해서 진행
MovieLens_pivot = pd.DataFrame(MovieLens_pivot).iloc[:50, :50]

In [20]:
# 변수 초기화
size = np.size(MovieLens_pivot, axis = 0)
simPIP = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성

# for문을 통한 각 유저별 PIP Similarity 측정
for u, i in zip(MovieLens_pivot.index, range(len(MovieLens_pivot))):
    for v, j in zip(MovieLens_pivot.index, range(len(MovieLens_pivot))):
        #print(u, i, v, j)
        sim = PIP(u, v)
        simPIP[i][j] = sim
        simPIP[j][i] = sim
                    
# 측정된 값을 MinMaxScaler를 이용해 정규화                
min_max_scaler = preprocessing.MinMaxScaler()
simPIP = min_max_scaler.fit_transform(simPIP)
df_simPIP = pd.DataFrame(index = MovieLens_pivot.index, columns = MovieLens_pivot.index, data = simPIP)
df_simPIP

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.978165,0.457508,0.622228,0.401543,0.211453,0.376502,0.825878,0.277024,0.044915,0.670849,...,0.206933,0.623965,0.552336,0.317003,0.564416,0.605678,0.691385,0.319204,0.415037,0.644412
1,0.507291,1.000000,0.457283,0.258178,0.211889,0.465105,0.489068,0.186387,0.070516,0.513851,...,0.249230,0.400818,0.380992,0.390855,0.477824,0.374346,0.315059,0.422078,0.396268,0.433406
2,1.000000,0.659258,1.000000,0.578900,0.602419,0.833143,0.858063,0.495596,0.014133,1.000000,...,0.423377,0.933255,0.881542,0.371023,0.837007,0.882784,0.999111,0.647750,0.593859,1.000000
3,0.605774,0.356438,0.539360,1.000000,0.583407,0.539902,0.728938,0.220981,0.142616,0.532885,...,0.284290,0.614548,0.496333,0.130326,0.633056,0.523844,0.678460,0.462126,0.165337,0.602590
4,0.138826,0.106743,0.241723,0.216421,1.000000,0.337035,0.352392,0.050748,0.173405,0.269327,...,0.003354,0.240486,0.282790,0.274122,0.294032,0.191075,0.305719,0.207414,0.171472,0.391657
5,0.128519,0.136783,0.237863,0.121484,0.158163,1.000000,0.192597,0.060193,0.018989,0.263373,...,0.096771,0.242250,0.209705,0.190450,0.215467,0.235897,0.250555,0.252822,0.251775,0.267035
6,0.181738,0.061337,0.146841,0.086176,0.057204,0.130899,1.000000,0.054768,0.142406,0.159844,...,0.000000,0.165789,0.132195,0.132787,0.153773,0.125455,0.223627,0.060881,0.101053,0.155977
7,0.345036,0.224815,0.353796,0.183289,0.214092,0.419232,0.684822,1.000000,0.102839,0.364182,...,0.165696,0.370901,0.338676,0.259492,0.377989,0.343244,0.719878,0.232273,0.348293,0.417354
8,0.000000,0.021581,0.000000,0.109853,0.414006,0.000000,0.853381,0.019916,1.000000,0.000000,...,0.059265,0.000000,0.000000,0.488387,0.000000,0.000000,0.000000,0.000000,0.226681,0.000000
9,0.637264,0.426346,0.609511,0.323427,0.362540,0.558645,0.550010,0.248790,0.015071,0.948280,...,0.231614,0.631202,0.507161,0.250782,0.564538,0.622394,0.744413,0.459448,0.352197,0.734941


# 2. Singularity

### 2.1 Singularity의 동기

RS에서 CF의 핵심은 추천하고자하는 사용자에게 k-neighbors의 선택이다. 이 과정은 흔히 이용 가능한 모든 정보를 이용하지 않는 유사성 측정을 사용함으로써 달성되는 경우가 많으며, 더욱이 이용되는 정보는 예상한 것보다 훨씬 덜 목적적합하게 된다. rating값을 분류하는 것뿐만 아니라 상황적 맥락정보(contextual information, 이용자 전체에서 도출)를 활용한 결과의 개선을 도모할 것이다.

### 2.2 Contextual information의 의미

앞선 전통적인 유사도 측정은 두 사용자가 레이팅한 값만을 고려하여 계산된다. 대부분 효율적으로 계산되지만, 유저들에 의해 만들어진 레이팅에 대한 맥락은 무시된다. Singularity는 유저가 만든 각 항목의 등급을 분석하여 이러한 맥락을 고려한다. 
만약 두 명의 유저가 투표한 표가 한 항목에 대해 유사하다면, 이는 해당 유저와 나머지 사용자 간에 찾기 어려운 유사성을 나타내기 때문에 다른 사용자가 투표한 항목이 서로 다를 경우 우리는 더 많은 가치를 갖는다. 한편, 대다수의 유저가 유사한 방식으로 어떤 항목에 대해 투표했다면, 나머지 유저와 동일한 방식으로 이 항목에 대해 투표한 두 유저 간의 유사성은 그다지 관련이 없다고 보아야 한다.

### 2.3 Singularity의 개념의 통합

Singularity는 두 유저에게 할당된 유사성에 대한 항목의 기여는 절대적으로 간주되는 것이 아니라 데이터 전체 나머지 유저가 항목에 레이팅한 값을 비교하여 고려되어야 한다는 것이다. 특이한 유사성이 정상적인 유사성보다 더 높은 값을 부여하는 방식으로 유사성의 가치를 특이성 값으로 변조한다는 것이다. 

### 2.4 Basic model 구현 

U: user의 전체 집합, I: item의 전체 집합 <br>
R: 긍정의 relevence 집합(MovieLens Data에서는 4, 5)<br>
P_i: 아이템 i에 대하여 긍정적으로 rating한 user(threshold = 4)<br>
N_i: 아이템 i에 대하여 부정적으로 rating한 user<br>
S_pi: 1 - len(P_i) / len(U) <br>
S_ni: 1 - len(N_i) / len(U) <br>
S_pi + S_ni = 1

![image.png](setcase.JPG)

위 표에 따라 user1, user2의 rating을 보고 집합 A, B, C를 할당한다.

In [21]:
# 변수 초기화
threshold = 4 # 4점 이상의 레이팅을 positive로 보겠음
R = [4, 5] # positive rating value
Rc = [1, 2, 3] # non-positive rating value
singularity = OrderedDict()
P = OrderedDict()
N = OrderedDict()
Sp = OrderedDict()
Sn = OrderedDict()
items = MovieLens_pivot.columns
users = MovieLens_pivot.index

In [22]:
# user, item에 해당하는 rating값 반환
def getRating(user, item):
    return MovieLens_pivot.loc[user][item]

In [23]:
# P_i, N_i
for item in items:
    pvalues = []
    nvalues = []
    for user in users:
        if MovieLens_pivot.loc[user][item] in R:
            pvalues.append(user)
        else:
            nvalues.append(user)
    P[item] = pvalues
    N[item] = nvalues

In [24]:
# S_pi, S_ni
for item in items:
    value = 1 - len(P[item]) / len(MovieLens_pivot)
    Sp[item] = value
    Sn[item] = 1 - value

In [25]:
# user1, user2의 singularity similarity 측정
def Singularity(user1, user2):
    # P_i, N_i
    P = OrderedDict()
    N = OrderedDict()
    for item in items:
        pvalues = []
        nvalues = []
        for user in users:
            if MovieLens_pivot.loc[user][item] in R:
                pvalues.append(user)
            else:
                nvalues.append(user)
        P[item] = pvalues
        N[item] = nvalues
        
    # S_pi, S_ni
    Sp = OrderedDict()
    Sn = OrderedDict()
    for item in items:
        value = 1 - len(P[item]) / len(MovieLens_pivot)
        Sp[item] = value
        Sn[item] = 1 - value    
    
    #similarity    
    A = np.array([])
    B = np.array([])
    C = np.array([])
    for item in items:
        rating1 = getRating(user1, item)
        rating2 = getRating(user2, item)
        if (rating1 in R) and (rating2 in R):
            singularity[item] = Sp[item]**2
            A = np.append(A, item)
        elif (rating1 in Rc) and (rating2 in Rc):
            singularity[item] = Sn[item]**2
            B = np.append(B, item)
        else:
            singularity[item] = Sp[item] * Sn[item]
            C = np.append(C, item)
    simSING = ((sum([((1 - (getRating(user1, item) - getRating(user2, item))**2) * Sp[item]**2) for item in A]) / len(A)
               if len(A) > 0 else 0)
             + (sum([((1 - (getRating(user1, item) - getRating(user2, item))**2) * Sn[item]**2) for item in B]) / len(B) 
               if len(B) > 0 else 0)
            + (sum([((1 - (getRating(user1, item) - getRating(user2, item))**2) * Sp[item] * Sn[item]) for item in C]) / len(C) 
               if len(C) > 0 else 0)) / 3.0
    
    return simSING

### 2.5 MovieLense Data를 이용하여 Singularity Similarity 측정

In [26]:
# 결과 행렬값 초기화
size = np.size(MovieLens_pivot, axis = 0)
simSING = np.zeros(shape = (size, size)) # 0으로 초기화 된 행렬 생성

# for문을 통한 각 유저별 Singularity Similarity 측정
for u, i in zip(MovieLens_pivot.index, range(len(MovieLens_pivot))):
    for v, j in zip(MovieLens_pivot.index, range(len(MovieLens_pivot))):
        #print(u, i, v, j)
        sim = Singularity(u, v)
        simSING[i][j] = sim
        simSING[j][i] = sim

# 측정된 값을 MinMaxScaler를 이용해 정규화  
min_max_scaler = preprocessing.MinMaxScaler()
simSING = min_max_scaler.fit_transform(simSING)
df_simSING = pd.DataFrame(index = MovieLens_pivot.index, columns = MovieLens_pivot.index, data = simSING)
df_simSING

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.000000,0.549786,0.802935,0.614681,0.666466,0.424066,0.698058,0.668103,0.316642,0.646062,...,0.502419,0.628167,0.631860,0.346234,0.451857,0.500929,0.541015,0.275901,0.349516,0.543504
1,0.544405,1.000000,0.649104,0.638274,0.525219,0.340120,0.471370,0.423639,0.314801,0.493511,...,0.272504,0.510624,0.254318,0.316452,0.238951,0.341754,0.488468,0.371759,0.226978,0.571093
2,0.485157,0.257320,1.000000,0.537398,0.442342,0.230151,0.392508,0.329774,0.282983,0.541530,...,0.228262,0.609972,0.664987,0.118721,0.229650,0.255889,0.317111,0.078688,0.214385,0.277020
3,0.516307,0.547465,0.767402,1.000000,0.507466,0.304139,0.448358,0.377334,0.312669,0.577405,...,0.000000,0.751613,0.410200,0.323436,0.444963,0.477701,0.409027,0.423831,0.401499,0.527639
4,0.707364,0.590613,0.821526,0.663356,1.000000,0.528209,0.601571,0.526797,0.480127,0.686974,...,0.493077,0.808899,0.647585,0.499188,0.627731,0.642869,0.611259,0.562015,0.517475,0.798823
5,0.439671,0.368023,0.656290,0.445783,0.465889,1.000000,0.472432,0.416829,0.509587,0.489782,...,0.375657,0.421534,0.431696,0.408096,0.568788,0.532959,0.591806,0.322757,0.275431,0.635607
6,0.765481,0.593823,0.830401,0.663876,0.646415,0.589768,1.000000,0.571434,0.497552,0.788425,...,0.387788,0.675110,0.704535,0.587438,0.778754,0.700600,0.792561,0.489190,0.628642,0.790736
7,0.679100,0.454624,0.720909,0.514326,0.481097,0.428428,0.471881,1.000000,0.458576,0.676650,...,0.373653,0.568280,0.576248,0.353892,0.578178,0.599157,0.670168,0.248618,0.503284,0.599604
8,0.213567,0.214206,0.596043,0.333573,0.315457,0.421342,0.260719,0.346929,1.000000,0.445930,...,0.357122,0.269803,0.274375,0.434507,0.328359,0.044149,0.530603,0.483697,0.463923,0.326725
9,0.545905,0.376155,0.764081,0.570218,0.533983,0.344360,0.624705,0.563167,0.406669,1.000000,...,0.212124,0.610445,0.591167,0.273434,0.325749,0.488769,0.518988,0.207729,0.317580,0.501654
